<a href="https://colab.research.google.com/github/Gauravhulmukh/Chatbot-Development-for-Regional-language-using-Artificial-Intelligence/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pickle
import json
import random

# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#importing dataset
import json
with open('/content/drive/My Drive/Colab Notebooks/intents.json') as json_data:
  intents = json.load(json_data)

In [6]:
#simply creating array
words = []
classes = []
documents = []
ignore_words = ['?']#it will ignore  question MARK from pattern

#create loop intents->patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #seperate words:hi there=hi,there
        w = nltk.word_tokenize(pattern)
        # add seperated words from each pattern into words array
        words.extend(w)
        #create set of words(tokenize_pattern) + tag =Documents
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))
print (len(documents),documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

22 [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thanks', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['how', 'to', 'register', 'payment', '?'], 'paymentregistration'), (['what', 'I', 'need', 'to', 'need', 'to', 'create', 'payment', '?'], 'paymentregistration'), (['How', 'to', 'enter', 'new', 'payment', '?'], 'paymentregistration'), (['Please', 'help', 'to', 'validate', 'payment', '?'], 'paymentvalidate'), (['Is', 'payment', 'valid', '?'], 'paymentvalidate'), (['What', 'needs', 'to', 'be', 'done', 'for', 'invoice', 'payment', 'validation', '?'], 'paymentvalidate'), (['How', 'to', 'pay', 'invoice', '?'], 'startpayment'), (['I', 'would', 'to', 'submit', 'invoice', 'for', 'payment'], 'startpayment'), (['How', 'it', 'works', 'to', 'pay', '?'], 'start

In [0]:
#creating arrays for training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        #it is comparing alphabhetical ordered array of words with non-alphabhetical ordered array of stem words of document
       #match 1 else 0     
        
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #it is comparing classes array & doc[1] array

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [8]:
#Clears the default graph stack and resets the global default graph.
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 2999  | total loss: 1.58223 | time: 0.008s
| Adam | epoch: 1000 | loss: 1.58223 - acc: 0.8253 -- iter: 16/22
Training Step: 3000  | total loss: 2.04141 | time: 0.011s
| Adam | epoch: 1000 | loss: 2.04141 - acc: 0.7761 -- iter: 22/22
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
pickle.dump({
    'words': words,
    'classes': classes,
    'train_x': train_x,
    'train_y': train_y
    }, open('training_data', 'wb'))